In [1]:
import numpy as np
import pandas as pd
import os
import re

# Data by season

In [2]:
file_loc = 'C:/Users/markchen/Desktop/高胜率玩家分析/data/raw_data/'
file_name = '玩家投注资料_2020_Q1.csv'
raw_data = pd.read_csv(file_loc + file_name)

C:\Users\markchen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Preprocess

## Groupby

In [7]:
player_win_info = raw_data.groupby('MainAccountSN').agg(
    玩家季度获胜数 = ('BetResult', lambda x: sum(x == 'Won')),
    玩家季度胜率 = ('BetResult', lambda x: sum(x == 'Won') / len(x)),
    玩家投注单量 = ('TransactionMGUID', 'count'),
    # 赔率资讯
    投注赔率平均 = ('InitailBetRate', 'mean'),
    投注赔率中位数 = ('InitailBetRate', 'median'),
    投注赔率标准差 = ('InitailBetRate', 'std'),
    # 比赛资讯,
    投注比赛数 = ('EventCode', lambda x: x.nunique()),
).reset_index()
player_win_info.fillna(0, inplace = True)

## Split

### 注单量 Q1 ~ Pr 95

In [8]:
q1_bet_num = player_win_info.玩家投注单量.quantile(0.25)
pr95_bet_num = player_win_info.玩家投注单量.quantile(0.95)
player_bet_lower_part_2 = player_win_info.query('@q1_bet_num <= 玩家投注单量 <= @pr95_bet_num').reset_index(drop = True)

In [9]:
# rr = 92
player_bet_lower_part_2['true_win_rate_92_mean'] = 1 / player_bet_lower_part_2['投注赔率平均'] * 0.92
player_bet_lower_part_2['true_win_rate_92_median'] = 1 / player_bet_lower_part_2['投注赔率中位数'] * 0.92
# rr = 95
player_bet_lower_part_2['true_win_rate_95_mean'] = 1 / player_bet_lower_part_2['投注赔率平均'] * 0.95
player_bet_lower_part_2['true_win_rate_95_median'] = 1 / player_bet_lower_part_2['投注赔率中位数'] * 0.95
col_names_list = list(player_bet_lower_part_2.filter(like = 'true_win_rate').columns)
col_names_list_provided = [_ + '_increment' for _ in col_names_list]
increment_df = player_bet_lower_part_2[['玩家季度胜率']].values / player_bet_lower_part_2.filter(like = 'true_win_rate') - 1
increment_df.columns = col_names_list_provided
player_bet_mid = pd.concat([player_bet_lower_part_2, increment_df], axis = 1)

### Second split

In [10]:
# Split
q2_odd_std = player_bet_mid.投注赔率标准差.quantile(0.5)
player_bet_mid_std_high = player_bet_mid.query('@q2_odd_std <= 投注赔率标准差').reset_index(drop = True)
player_bet_mid_std_low = player_bet_mid.query('投注赔率标准差 < @q2_odd_std').reset_index(drop = True)

In [11]:
## Low variance
bet_num_mid_std_low = player_bet_mid_std_low.玩家投注单量.quantile(0.5)
game_num_mid_std_low = player_bet_mid_std_low.投注比赛数.quantile(0.5)
target_index = player_bet_mid_std_low.true_win_rate_95_mean_increment.quantile(0.95)
bet_mid_std_low_df = player_bet_mid_std_low.query('玩家投注单量 > @bet_num_mid_std_low and \
                              投注比赛数 >= @game_num_mid_std_low and \
                              true_win_rate_95_mean_increment >= @target_index')
# High variance
bet_num_mid_std_high = player_bet_mid_std_high.玩家投注单量.quantile(0.5)
game_num_mid_std_high = player_bet_mid_std_high.投注比赛数.quantile(0.5)
target_index_high = player_bet_mid_std_high.true_win_rate_95_median_increment.quantile(0.95)
bet_mid_std_high_df = player_bet_mid_std_high.query('玩家投注单量 > @bet_num_mid_std_high and \
                              投注比赛数 >= @game_num_mid_std_high and \
                              true_win_rate_95_median_increment >= @target_index')
# Final result bet mid
bet_mid_account_list = bet_mid_std_high_df.MainAccountSN.to_list() + bet_mid_std_low_df.MainAccountSN.to_list()

## 注单量 > PR 95

In [ ]:
pr95_bet_num = player_win_info.玩家投注单量.quantile(0.95)
player_bet_lower_part_3 = player_win_info.query('@pr95_bet_num < 玩家投注单量 ').reset_index(drop = True)

In [ ]:
# rr = 93
player_bet_lower_part_3['true_win_rate_93_mean'] = 1 / player_bet_lower_part_3['投注赔率平均'] * 0.93
player_bet_lower_part_3['true_win_rate_93_median'] = 1 / player_bet_lower_part_3['投注赔率中位数'] * 0.93
# rr = 95
player_bet_lower_part_3['true_win_rate_95_mean'] = 1 / player_bet_lower_part_3['投注赔率平均'] * 0.95
player_bet_lower_part_3['true_win_rate_95_median'] = 1 / player_bet_lower_part_3['投注赔率中位数'] * 0.95
col_names_list = list(player_bet_lower_part_3.filter(like = 'true_win_rate').columns)
col_names_list_provided = [_ + '_increment' for _ in col_names_list]
increment_df = player_bet_lower_part_3[['玩家季度胜率']].values / player_bet_lower_part_3.filter(like = 'true_win_rate') - 1
increment_df.columns = col_names_list_provided
player_bet_high = pd.concat([player_bet_lower_part_3, increment_df], axis = 1)

### Second split

In [ ]:
q2_odd_std = player_bet_high.投注赔率标准差.quantile(0.5)
player_bet_high_std_high = player_bet_high.query('@q2_odd_std <= 投注赔率标准差').reset_index(drop = True)
player_bet_high_std_low = player_bet_high.query('投注赔率标准差 < @q2_odd_std').reset_index(drop = True)

In [ ]:
# Low variance
bet_num_high_std_low = player_bet_high_std_low.玩家投注单量.quantile(0.5)
game_num_high_std_low = player_bet_high_std_low.投注比赛数.quantile(0.5)
target_index = player_bet_high_std_low.true_win_rate_95_mean_increment.quantile(0.95)
bet_high_std_low_df = player_bet_high_std_low.query('玩家投注单量 > @bet_num_high_std_low and \
                              投注比赛数 >= @game_num_high_std_low and \
                              true_win_rate_95_mean_increment >= @target_index')
# High variance
bet_num_high_std_high = player_bet_high_std_high.玩家投注单量.quantile(0.5)
game_num_high_std_high = player_bet_high_std_high.投注比赛数.quantile(0.5)
target_index_high = player_bet_high_std_high.true_win_rate_95_median_increment.quantile(0.95)
bet_high_std_high_df = player_bet_high_std_high.query('玩家投注单量 > @bet_num_high_std_high and \
                              投注比赛数 >= @game_num_high_std_high and \
                              true_win_rate_95_median_increment >= @target_index')
# Final result bet high
bet_high_account_list = bet_high_std_low_df.MainAccountSN.to_list() + bet_high_std_high_df.MainAccountSN.to_list()

# Wrap-up

In [18]:
# Data by season
file_loc = 'C:/Users/markchen/Desktop/高胜率玩家分析/data/raw_data/'
file_name = '玩家投注资料_2020_Q1.csv'
raw_data = pd.read_csv(file_loc + file_name)

C:\Users\markchen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [19]:
def get_high_winning_player_AccountId(raw_data):
    player_win_info = raw_data.groupby('MainAccountSN').agg(
        玩家季度获胜数 = ('BetResult', lambda x: sum(x == 'Won')),
        玩家季度胜率 = ('BetResult', lambda x: sum(x == 'Won') / len(x)),
        玩家投注单量 = ('TransactionMGUID', 'count'),
        # 赔率资讯
        投注赔率平均 = ('InitailBetRate', 'mean'),
        投注赔率中位数 = ('InitailBetRate', 'median'),
        投注赔率标准差 = ('InitailBetRate', 'std'),
        # 比赛资讯,
        投注比赛数 = ('EventCode', lambda x: x.nunique()),).reset_index()
    player_win_info.fillna(0, inplace = True)
    ### 注单量 Q1 ~ Pr 95
    q1_bet_num = player_win_info.玩家投注单量.quantile(0.25)
    pr95_bet_num = player_win_info.玩家投注单量.quantile(0.95)
    player_bet_lower_part_2 = player_win_info.query('@q1_bet_num <= 玩家投注单量 <= @pr95_bet_num').reset_index(drop = True)
    # rr = 92
    player_bet_lower_part_2['true_win_rate_92_mean'] = 1 / player_bet_lower_part_2['投注赔率平均'] * 0.92
    player_bet_lower_part_2['true_win_rate_92_median'] = 1 / player_bet_lower_part_2['投注赔率中位数'] * 0.92
    # rr = 95
    player_bet_lower_part_2['true_win_rate_95_mean'] = 1 / player_bet_lower_part_2['投注赔率平均'] * 0.95
    player_bet_lower_part_2['true_win_rate_95_median'] = 1 / player_bet_lower_part_2['投注赔率中位数'] * 0.95
    col_names_list = list(player_bet_lower_part_2.filter(like = 'true_win_rate').columns)
    col_names_list_provided = [_ + '_increment' for _ in col_names_list]
    increment_df = player_bet_lower_part_2[['玩家季度胜率']].values / player_bet_lower_part_2.filter(like = 'true_win_rate') - 1
    increment_df.columns = col_names_list_provided
    player_bet_mid = pd.concat([player_bet_lower_part_2, increment_df], axis = 1)
    ### Second split
    # Split
    q2_odd_std = player_bet_mid.投注赔率标准差.quantile(0.5)
    player_bet_mid_std_high = player_bet_mid.query('@q2_odd_std <= 投注赔率标准差').reset_index(drop = True)
    player_bet_mid_std_low = player_bet_mid.query('投注赔率标准差 < @q2_odd_std').reset_index(drop = True)
    ## Low variance
    bet_num_mid_std_low = player_bet_mid_std_low.玩家投注单量.quantile(0.5)
    game_num_mid_std_low = player_bet_mid_std_low.投注比赛数.quantile(0.5)
    target_index = player_bet_mid_std_low.true_win_rate_95_mean_increment.quantile(0.95)
    bet_mid_std_low_df = player_bet_mid_std_low.query('玩家投注单量 > @bet_num_mid_std_low and \
                                  投注比赛数 >= @game_num_mid_std_low and \
                                  true_win_rate_95_mean_increment >= @target_index')
    # High variance
    bet_num_mid_std_high = player_bet_mid_std_high.玩家投注单量.quantile(0.5)
    game_num_mid_std_high = player_bet_mid_std_high.投注比赛数.quantile(0.5)
    target_index_high = player_bet_mid_std_high.true_win_rate_95_median_increment.quantile(0.95)
    bet_mid_std_high_df = player_bet_mid_std_high.query('玩家投注单量 > @bet_num_mid_std_high and \
                                  投注比赛数 >= @game_num_mid_std_high and \
                                  true_win_rate_95_median_increment >= @target_index')
    # Final result bet mid
    bet_mid_account_list = bet_mid_std_high_df.MainAccountSN.to_list() + bet_mid_std_low_df.MainAccountSN.to_list()

    ## 注单量 > PR 95
    pr95_bet_num = player_win_info.玩家投注单量.quantile(0.95)
    player_bet_lower_part_3 = player_win_info.query('@pr95_bet_num < 玩家投注单量 ').reset_index(drop = True)
    # rr = 93
    player_bet_lower_part_3['true_win_rate_93_mean'] = 1 / player_bet_lower_part_3['投注赔率平均'] * 0.93
    player_bet_lower_part_3['true_win_rate_93_median'] = 1 / player_bet_lower_part_3['投注赔率中位数'] * 0.93
    # rr = 95
    player_bet_lower_part_3['true_win_rate_95_mean'] = 1 / player_bet_lower_part_3['投注赔率平均'] * 0.95
    player_bet_lower_part_3['true_win_rate_95_median'] = 1 / player_bet_lower_part_3['投注赔率中位数'] * 0.95
    col_names_list = list(player_bet_lower_part_3.filter(like = 'true_win_rate').columns)
    col_names_list_provided = [_ + '_increment' for _ in col_names_list]
    increment_df = player_bet_lower_part_3[['玩家季度胜率']].values / player_bet_lower_part_3.filter(like = 'true_win_rate') - 1
    increment_df.columns = col_names_list_provided
    player_bet_high = pd.concat([player_bet_lower_part_3, increment_df], axis = 1)
    ### Second split
    q2_odd_std = player_bet_high.投注赔率标准差.quantile(0.5)
    player_bet_high_std_high = player_bet_high.query('@q2_odd_std <= 投注赔率标准差').reset_index(drop = True)
    player_bet_high_std_low = player_bet_high.query('投注赔率标准差 < @q2_odd_std').reset_index(drop = True)
    # Low variance
    bet_num_high_std_low = player_bet_high_std_low.玩家投注单量.quantile(0.5)
    game_num_high_std_low = player_bet_high_std_low.投注比赛数.quantile(0.5)
    target_index = player_bet_high_std_low.true_win_rate_95_mean_increment.quantile(0.95)
    bet_high_std_low_df = player_bet_high_std_low.query('玩家投注单量 > @bet_num_high_std_low and \
                                  投注比赛数 >= @game_num_high_std_low and \
                                  true_win_rate_95_mean_increment >= @target_index')
    # High variance
    bet_num_high_std_high = player_bet_high_std_high.玩家投注单量.quantile(0.5)
    game_num_high_std_high = player_bet_high_std_high.投注比赛数.quantile(0.5)
    target_index_high = player_bet_high_std_high.true_win_rate_95_median_increment.quantile(0.95)
    bet_high_std_high_df = player_bet_high_std_high.query('玩家投注单量 > @bet_num_high_std_high and \
                                  投注比赛数 >= @game_num_high_std_high and \
                                  true_win_rate_95_median_increment >= @target_index')
    # Final result bet high
    bet_high_account_list = bet_high_std_low_df.MainAccountSN.to_list() + bet_high_std_high_df.MainAccountSN.to_list()
    Export_list = bet_mid_account_list + bet_high_account_list
    return Export_list

In [ ]:
# Iterate

In [26]:
file_loc = 'C:/Users/markchen/Desktop/高胜率玩家分析/data/raw_data/'
file_list = os.listdir(file_loc)

In [27]:
total_account_list = []
for file_name in file_list:
    raw_data = pd.read_csv(file_loc + file_name)
    season_account_list = get_high_winning_player_AccountId(raw_data)
    total_account_list.append(season_account_list)

C:\Users\markchen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [28]:
total_account_list

[[7313932,
  8841618,
  545065,
  6534514,
  8491244,
  9389921,
  9909096,
  2331915,
  2533693,
  5002333,
  7094493,
  7725224,
  2725510,
  3102358],
 [6234475, 6905245, 7109147, 9671655, 10291827, 3429454, 5850443],
 [10593387, 10768333, 2336759, 2421517, 2434159, 8499126, 9958767, 10704721],
 [11327292, 11549257, 2159851, 5393346, 6359473, 10705671, 10330122]]

In [62]:
index_df = pd.DataFrame(total_account_list, index = ['Q1', 'Q2', 'Q3', 'Q4'])

In [63]:
index_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
Q1,7313932,8841618,545065,6534514,8491244,9389921,9909096,2331915.0,2533693.0,5002333.0,7094493.0,7725224.0,2725510.0,3102358.0
Q2,6234475,6905245,7109147,9671655,10291827,3429454,5850443,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q3,10593387,10768333,2336759,2421517,2434159,8499126,9958767,10704721.0,NaN,NaN,NaN,NaN,NaN,NaN
Q4,11327292,11549257,2159851,5393346,6359473,10705671,10330122,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
flatten_total_account_list = sum(total_account_list, [])

In [32]:
len(flatten_total_account_list)

36

In [31]:
len(set(flatten_total_account_list))

36

# Players' seasonal performance 

In [38]:
file_loc = 'C:/Users/markchen/Desktop/高胜率玩家分析/data/raw_data/'
file_name = '玩家投注资料_2020_year.csv'
bet_total_df = pd.read_csv(file_loc + file_name, parse_dates=['BetDatetime'])

C:\Users\markchen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [53]:
target_df = bet_total_df.query('MainAccountSN in @flatten_total_account_list').reset_index(drop = True)

In [54]:
target_df['Quarter_time'] = target_df.BetDatetime.dt.to_period("Q")

In [64]:
target_df_groupby = target_df.groupby(['MainAccountSN', 'Quarter_time']).agg(
    玩家总投注额 = ('RealBetAmount_CNY', 'sum'),
    玩家总损益 = ('CustomerWinLoss_CNY', 'sum'),
    玩家投注单量 = ('TransactionMGUID', 'count'),
    玩家季度胜率 = ('BetResult', lambda x: sum(x == 'Won') / len(x)),
    # 赔率资讯
    投注赔率平均 = ('InitailBetRate', 'mean'),
    投注赔率中位数 = ('InitailBetRate', 'median'),
    投注赔率标准差 = ('InitailBetRate', 'std'),
).reset_index()

In [65]:
# rr = 95
target_df_groupby['true_win_rate_95_mean'] = 1 / target_df_groupby['投注赔率平均'] * 0.95
target_df_groupby['true_win_rate_95_median'] = 1 / target_df_groupby['投注赔率中位数'] * 0.95

In [66]:
col_names_list = list(target_df_groupby.filter(like = 'true_win_rate').columns)
col_names_list_provided = [_ + '_increment' for _ in col_names_list]
increment_df = target_df_groupby[['玩家季度胜率']].values / target_df_groupby.filter(like = 'true_win_rate') - 1
increment_df.columns = col_names_list_provided

In [67]:
target_df_final = pd.concat([target_df_groupby, increment_df], axis = 1)

In [68]:
target_df_final

,MainAccountSN,Quarter_time,玩家总投注额,玩家总损益,玩家投注单量,玩家季度胜率,投注赔率平均,投注赔率中位数,投注赔率标准差,true_win_rate_95_mean,true_win_rate_95_median,true_win_rate_95_mean_increment,true_win_rate_95_median_increment
0,545065,2020Q1,17340.0,18099.00,44,0.772727,2.683182,2.75,0.540967,0.354057,0.345455,1.182492,1.236842
1,2159851,2020Q1,6889.0,-189.00,5,0.600000,2.074000,1.93,0.379776,0.458052,0.492228,0.309895,0.218947
2,2159851,2020Q2,104157.0,-70916.75,57,0.210526,2.130877,1.90,0.570273,0.445826,0.500000,-0.527783,-0.578947
3,2159851,2020Q3,1550.0,-1198.00,4,0.250000,1.612500,1.58,0.195512,0.589147,0.601266,-0.575658,-0.584211
4,2159851,2020Q4,44406.0,33148.17,32,0.937500,2.014063,1.93,0.424255,0.471683,0.492228,0.987562,0.904605
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,10705671,2020Q4,321120.0,252468.00,20,0.950000,1.803500,1.58,0.510617,0.526754,0.601266,0.803500,0.580000
89,10768333,2020Q3,32432.0,23647.00,47,0.489362,4.045106,4.45,1.691358,0.234852,0.213483,1.083705,1.292273
90,10768333,2020Q4,30824.0,-8259.76,83,0.325301,3.147229,2.75,1.781319,0.301853,0.345455,0.077681,-0.058339
91,11327292,2020Q4,11652.0,10052.50,51,0.705882,3.092157,2.44,1.744240,0.307229,0.389344,1.297578,0.813003


In [69]:
target_df_final.to_csv('筛选结果含赔率.csv', index = False, encoding = 'utf-8-sig')

In [61]:
target_df_groupby.to_csv('筛选结果.csv', index = False, encoding = 'utf-8-sig')